In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from cfd_common import *
from math import *
import scipy.interpolate
import os

In [ ]:
%matplotlib qt

In [ ]:
plt.ion()

In [ ]:
path_prefix = 'validation_naiveDrD-noncentered_a0.5_dt0.0001_50x50'
if not os.path.exists(path_prefix):
    os.mkdir(path_prefix)

In [ ]:
##### Domain #####

# Domain size
Lx = pi
Ly = pi

# Physical grid size
Nx_phys = 50
Ny_phys = 50
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys

# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

In [ ]:
a = 0.5
b = 0

def exact_solution (x, y, t):
    u =  np.sin(x) * np.cos(y) * np.cos(t)
    v = -np.sin(y) * np.cos(x) * np.cos(t)
    ρ = 1 + a * np.sin(x)**2 * np.sin(y)**2 + b * np.sin(x)**4 * np.sin(y)**4
    p = -2 * np.cos(x) * np.cos(y) * np.cos(t)
    fx = ((-np.sin(t)*np.cos(y) + np.cos(t)**2*np.cos(x))*(b*np.sin(x)**4*np.sin(y)**4 + a**np.sin(x)**2*np.sin(y)**2 + 1) + 4*np.cos(t)*np.cos(y))*np.sin(x)
    fy = (np.sin(t)*np.cos(x) + np.cos(t)**2*np.cos(y))*(b*np.sin(x)**4*np.sin(y)**4 + a*np.sin(x)**2*np.sin(y)**2 + 1)*np.sin(y)
    return ρ,u,v,p

def force (x, y, t):
    fx = ((-np.sin(t)*np.cos(y) + np.cos(t)**2*np.cos(x))*(b*np.sin(x)**4*np.sin(y)**4 + a**np.sin(x)**2*np.sin(y)**2 + 1) + 4*np.cos(t)*np.cos(y))*np.sin(x)
    fy = (np.sin(t)*np.cos(x) + np.cos(t)**2*np.cos(y))*(b*np.sin(x)**4*np.sin(y)**4 + a*np.sin(x)**2*np.sin(y)**2 + 1)*np.sin(y)
    return fx,fy

In [ ]:
##### Fields #####

ex_ρ,ex_u,ex_v,ex_p = exact_solution(xx, yy, 0)

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
u[1:-1,1:-1] = np.transpose( ex_u )
v = np.zeros((Nx,Ny))
v[1:-1,1:-1] = np.transpose( ex_v )

# Pressure field phi[i,j] and its gradient (dx_phi,dy_phi)
phi = np.zeros((Nx,Ny))
phi[1:-1,1:-1] = np.transpose( ex_p )
dx_phi = np.zeros((Nx,Ny))
dy_phi = np.zeros((Nx,Ny))

# Density field
ρ = np.zeros((Nx,Ny))
ρ[1:-1,1:-1] = np.transpose( ex_ρ )

# Total mass
def mass_check (ρ):
    return Δx*Δy*np.sum(ρ)

In [ ]:
Δt = 0.0001

t = 0. # total time

# how much do we correct for diffusion in semi-lag advection at the price of dispersion
ρ_semilag2_coeff = 0.8
u_semilag2_coeff = 0.8

niter = 0
disp_modulo = 100

In [ ]:
#data = np.load("bubble-test-200x200-t2.40237-niter240237.npz")
#u = data['u']
#v = data['v']
#ρ = data['rho']
#t = 2.40237
#niter = 240237
# np.savez("bubble-test-200x200-.npz", u=u, v=v, rho=ρ)

In [ ]:
err_history_t = []
err_history_uabs = []
err_history_umean = []
err_history_vabs = []
err_history_vmean = []
err_history_ρabs = []
err_history_ρmean = []
err_history_pabs = []
err_history_pmean = []

In [ ]:
fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)

def do_plot (save_image):
    global err_history_t, err_history_uabs, err_history_umean, err_history_vabs, err_history_vmean, err_history_ρabs, err_history_ρmean, err_history_pabs, err_history_pmean
    fig.clear()
    ((ax1,ax2,ax3),(ax4,ax5,ax6)) = fig.subplots(nrows=2, ncols=3)

    def plot_vel (ax, modx=None, mody=None, vmax=3):
        if modx is None:
            modx = max(1,int(round(Nx_phys/20)))
        if mody is None:
            mody = max(1,int(round(Ny_phys/20)))
        ax.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(u[1:-1,1:-1])[::modx,::mody], np.transpose(v[1:-1,1:-1])[::modx,::mody], scale=vmax*5, minlength=0.001, headwidth=4, headlength=6, headaxislength=5, width=0.005)
        ax.set_aspect('equal', adjustable='box')

    fig.suptitle(r"Validation. SemiLag2({}) for $\vec{{u}}$, ${}\times{}$ grid. SemiLag2({}) for $\rho$, total mass : {:.2f}, $t={:.3f}$, $\Delta t=${:.0e}.".format(u_semilag2_coeff,Nx_phys,Ny_phys,ρ_semilag2_coeff,mass_check(ρ),t,Δt))
    
    p = np.transpose(phi[1:-1,1:-1])
    im = ax1.imshow(p-np.mean(p), origin='lower', extent=(0,Lx,0,Ly), cmap='plasma', vmin=-2.2, vmax=2.2)
    fig.colorbar(im, ax=ax1)
    ax1.axis('off')
    plot_vel(ax1)
    ax1.set_title("Pressure and velocity fields")

    ex_ρ,ex_u,ex_v,ex_p = exact_solution(xx, yy, t)
    err_u = np.transpose(u[1:-1, 1:-1]) - ex_u
    err_v = np.transpose(v[1:-1, 1:-1]) - ex_v
    err_ρ = np.transpose(ρ[1:-1, 1:-1]) - ex_ρ
    err_p = (p-np.mean(p)) - (ex_p-np.mean(ex_p))
    err_history_t += [ t ]
    err_history_uabs += [ np.max(np.abs(err_u)) ]
    err_history_umean += [ np.mean(np.abs(err_u)) ]
    err_history_vabs += [ np.max(np.abs(err_v)) ]
    err_history_vmean += [ np.mean(np.abs(err_v)) ]
    err_history_ρabs += [ np.max(np.abs(err_ρ)) ]
    err_history_ρmean += [ np.mean(np.abs(err_ρ)) ]
    err_history_pabs += [ np.max(np.abs(err_p)) ]
    err_history_pmean += [ np.mean(np.abs(err_p)) ]
    
    k = 1
    
    im = ax2.imshow(err_p, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.2, vmax=+k*0.2, cmap='bwr')
    fig.colorbar(im, ax=ax2)
    ax2.axis('off')
    ax2.set_title("Error on pressure")
    
    im = ax3.imshow(err_ρ, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.03, vmax=+k*0.03, cmap='bwr')
    fig.colorbar(im, ax=ax3)
    ax3.axis('off')
    ax3.set_title("Error on density")
    
    im = ax4.imshow(err_u, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.08, vmax=+k*0.08, cmap='bwr')
    fig.colorbar(im, ax=ax4)
    ax4.axis('off')
    ax4.set_title("Error on $u$")
    
    im = ax5.imshow(err_v, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.08, vmax=+k*0.08, cmap='bwr')
    fig.colorbar(im, ax=ax5)
    ax5.axis('off')
    ax5.set_title("Error on $v$")
    
    ax6.plot(err_history_t, err_history_uabs, label=r"$L^\infty$ err on $u$")
    ax6.plot(err_history_t, err_history_vabs, label=r"$L^\infty$ err on $v$")
    ax6.plot(err_history_t, err_history_ρabs, label=r"$L^\infty$ err on $\rho$")
    ax6.plot(err_history_t, np.array(err_history_pabs)/10, label=r"$0.1\times$ $L^\infty$ err on $p$")
#    ax6.set_ylim(0, 0.003)
    ax6.set_xlim(0, 3*pi)
    ax6.axvline(  pi/2, linestyle='--', color='gray', lw=1)
    ax6.axvline(  pi  , linestyle='-' , color='gray', lw=1)
    ax6.axvline(3*pi/2, linestyle='--', color='gray', lw=1)
    ax6.axvline(2*pi  , linestyle='-' , color='gray', lw=1)
    ax6.axvline(5*pi/2, linestyle='--', color='gray', lw=1)
    ax6.legend(loc='upper right', fontsize='small')
    
    fig.tight_layout()
    if save_image and niter != 0:
        fig.savefig(path_prefix+'/{:04d}.png'.format(niter//disp_modulo))
    fig.canvas.draw_idle()
    
    mpl_pause_background(0.00001)

for k in range(2):
    do_plot(False)

In [ ]:
LAP,LAP_solver = FD_2D_Laplacian_matrix (Nx_phys, Ny_phys, Δx, Δy, BCdir_left=False, BCdir_right=False, BCdir_top=False, BCdir_bot=False)

while loop_continue and t < 4*pi:

    def VelocityGhostPoints(u,v):
        ### left
        u[0, :] = -u[1, :]
        v[0, :] = v[1, :]
        ### right    
        u[-1, :] = -u[-2, :]
        v[-1, :] = v[-2, :]
        ### bottom
        u[:,  0] = u[:,  1]  # free slip
        v[:,  0] = -v[:,  1]  # imp.
        ### top     
        u[:, -1] = u[:, -2]
        v[:, -1] = -v[:, -2]
    
    VelocityGhostPoints(u,v)
    
    if u_semilag2_coeff is None:
        adv_u = SemiLag(u,v, u, Δx,Δy,Δt)
        adv_v = SemiLag(u,v, v, Δx,Δy,Δt)
    else:
        adv_u = SemiLag2(u,v, u, Δx,Δy,Δt, u_semilag2_coeff)
        adv_v = SemiLag2(u,v, v, Δx,Δy,Δt, u_semilag2_coeff)
 
    if ρ_semilag2_coeff is None:
        ρ = SemiLag(u,v, ρ, Δx,Δy,Δt)
    else:
        ρ = SemiLag2(u,v, ρ, Δx,Δy,Δt, ρ_semilag2_coeff)
    
    ρ[ 0, :] = 2 * (1) - ρ[ 1, :]
    ρ[-1, :] = 2 * (1) - ρ[-2, :]
    ρ[:,  0] = 2 * (1) - ρ[:,  1]
    ρ[:, -1] = 2 * (1) - ρ[:, -2]
    
    ustar = adv_u + Δt*Laplacian(u, Δx,Δy) / ρ
    vstar = adv_v + Δt*Laplacian(v, Δx,Δy) / ρ
    
    fx,fy = force(xx, yy, t)
    ustar[1:-1, 1:-1] += np.transpose(fx) * Δt / ρ[1:-1, 1:-1]
    vstar[1:-1, 1:-1] += np.transpose(fy) * Δt / ρ[1:-1, 1:-1]

    VelocityGhostPoints(ustar,vstar)
 
    divstar = Divergence(ustar,vstar, Δx,Δy)
    divstar /= Δt

    retry = True
    while retry:
        try:
            DρD,DρD_solver = FD_2D_DρD_matrix(Nx_phys, Ny_phys, Δx, Δy, ρ, BCdir_left=False, BCdir_right=False, BCdir_top=False, BCdir_bot=False, backend_build_C=True)
            retry = False
        except RuntimeError:
            ρ += np.random.normal(0,1e-10,size=(Nx,Ny))
    
    phi[1:-1, 1:-1] = DρD_solver(RHS=divstar[1:-1,1:-1])
    phi[0   ,  :  ] = + phi[ 1   ,  :  ]
    phi[  -1,  :  ] = + phi[-2   ,  :  ]
    phi[ :  ,   -1] = + phi[  :  ,   -2]
    phi[ :  , 0   ] = + phi[  :  ,    1]

    dx_phi[1:-1, :] = (phi[2:, :] - phi[:-2, :])/Δx/2
    dy_phi[:, 1:-1] = (phi[:, 2:] - phi[:, :-2])/Δy/2

    u = ustar - Δt*dx_phi / ρ
    v = vstar - Δt*dy_phi / ρ
    
    if niter%disp_modulo == 0:
        do_plot(save_image=True)
    
    t += Δt
    niter += 1

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(11,4))

fig.suptitle(r"Validation. SemiLag2({}) for $\vec{{u}}$, ${}\times{}$ grid. SemiLag2({}) for $\rho$, total mass : {:.2f}, $t={:.3f}$, $\Delta t=${:.0e}, $a={}$.".format(u_semilag2_coeff,Nx_phys,Ny_phys,ρ_semilag2_coeff,mass_check(ρ),t,Δt,a))

ax1.plot(err_history_t, err_history_uabs, label=r"$L^\infty$ err on $u$")
ax1.plot(err_history_t, err_history_vabs, label=r"$L^\infty$ err on $v$")
ax1.plot(err_history_t, err_history_ρabs, label=r"$L^\infty$ err on $\rho$")
ax1.plot(err_history_t, np.array(err_history_pabs)/10, label=r"$0.1\times$ $L^\infty$ err on $p$")
ax1.set_xlim(0, 3*pi)
ax1.axvline(  pi/2, linestyle='--', color='gray', lw=1)
ax1.axvline(  pi  , linestyle='-' , color='gray', lw=1)
ax1.axvline(3*pi/2, linestyle='--', color='gray', lw=1)
ax1.axvline(2*pi  , linestyle='-' , color='gray', lw=1)
ax1.axvline(5*pi/2, linestyle='--', color='gray', lw=1)
ax1.legend(loc='upper left', fontsize='small')
ax1.set_xlabel(r"$t$")

ax2.plot(err_history_t, err_history_umean, label=r"$L^1$ err on $u$")
ax2.plot(err_history_t, err_history_vmean, label=r"$L^1$ err on $v$")
ax2.plot(err_history_t, err_history_ρmean, label=r"$L^1$ err on $\rho$")
ax2.plot(err_history_t, np.array(err_history_pmean), label=r"$L^1$ err on $p$")
ax2.set_xlim(0, 3*pi)
ax2.axvline(  pi/2, linestyle='--', color='gray', lw=1)
ax2.axvline(  pi  , linestyle='-' , color='gray', lw=1)
ax2.axvline(3*pi/2, linestyle='--', color='gray', lw=1)
ax2.axvline(2*pi  , linestyle='-' , color='gray', lw=1)
ax2.axvline(5*pi/2, linestyle='--', color='gray', lw=1)
ax2.legend(loc='upper left', fontsize='small')
ax2.set_xlabel(r"$t$")

fig.tight_layout()
fig.savefig(path_prefix+'.err.pdf')

np.savez(path_prefix+'.npz', err_history_t=err_history_t, err_history_uabs=err_history_uabs, err_history_umean=err_history_umean, err_history_vabs=err_history_vabs, err_history_vmean=err_history_vmean, err_history_ρabs=err_history_ρabs, err_history_ρmean=err_history_ρmean, err_history_pabs=err_history_pabs, err_history_pmean=err_history_pmean)